#### `QR algorithm`

So far, we iterate based on

$$Q_{i+1}R_{i+1}=AQ_i$$

We can rearrange to do it slightly differently

$$Q_i^TQ_{i+1}R_{i+1}=Q_i^TAQ_i$$

If we denote $\tilde{Q}_{i+1}=Q_i^TQ_{i+1}$, and $Q_i^TAQ_i=A_i$, then, by construction, $\tilde{Q}_{i+1}$ is `orthogonal`, and $A_i$ is `similar` to $A$ and therefore, have the same eigenvalues

So we have

$$\tilde{Q}_{i+1}R_{i+1}=A_i$$

Now if we swap the factors, we get

$$\begin{align*}
R_{i+1}\tilde{Q}_{i+1}&=\left(Q^T_{i+1}AQ_i\right)\left(Q_i^TQ_{i+1}\right) \\
&=Q^T_{i+1}AQ_{i+1} \\
&=A_{i+1}
\end{align*}$$

So far, we haven't really changed anything to orthogonal iterations, just reformulate it at each step by regrouping and reusing the outcomes from orthogonal iterations

Therefore, if orthogonal iterations converge as $i\rightarrow \infty$, then

$$Q^TAQ=A_{i+1}$$

and $T$ in Schur decomposition is $A_{i+1}$

This indicates that instead of iterating over $Q$ as in orthogonal iterations, equivalently, we can also iterate over $T$ (in this case, $A$ itself)

This is the basic `QR algorithm` for finding eigenvalues of general matrices

(In practice, many other tricks would be needed, such as transformation into upper Hessenberg form)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def householder(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for i in range(n):
        x = R[i:, i]
        v = np.sign(x[0]) * np.linalg.norm(x) * np.eye(x.shape[0])[:,0] + x
        v /= np.linalg.norm(v)

        # Since all entries in R[i:, :i] are zero from previous iteration
        # applying transformation to R[i:, i:] would suffice
        R[i:, i:] -= 2 * np.outer(v, v) @ R[i:, i:]

        # If Q is needed explicitly
        Q[i:, :] -= 2 * np.outer(v, v) @ Q[i:, :]

    return Q.T, R

def back_substitution(T, i):
    n = T.shape[0]
    v_i = np.zeros(n)
    v_i[i] = 1
    T_modified = T[:i, :i] - np.eye(i) * T[i, i]

    for j in range(i-1, -1, -1):
        if T_modified[j, j] == 0:
            raise ValueError("Singular matrix")
        v_i[j] = -(T[j, j+1:i+1] @ v_i[j+1:i+1]) / T_modified[j, j]
    return v_i


def diagonalizable_mat(n):
    # Create diagonal matrix D with eigenvalues
    D = np.diag(np.concatenate((200*np.random.rand(n//2)-100, 0.1*np.random.rand(n-n//2))))

    # Generate a random invertible matrix
    P = np.random.rand(n, n)
    while np.linalg.cond(P) > 1e8:  # Check conditioning
        P = np.random.rand(n, n)

    # Use similarity transformation to create diagonalizable, but nonsymmetric matrix
    return P @ D @ np.linalg.inv(P)

In [ ]:
np.random.seed(50)

A_size = 8
A = diagonalizable_mat(A_size)
A_original = A.copy()
Q = np.eye(A.shape[0])
Q_final = np.eye(A.shape[0])

num_iter = 201

# QR algorithm
for i in range(num_iter):
    Q, R = householder(A)
    A = R @ Q
    Q_final = Q_final @ Q # Iterate to get Q in Schur form

    # Diagonal elements of A are approximation of eigenvalues
    if i % 40 == 0:
        print(np.diag(A))

v = []
for i in range(R.shape[0]):
    v_i = back_substitution(A, i)
    v_i /= np.linalg.norm(v_i)
    v.append(v_i)

v = np.column_stack(v)
print(f'\nEigenvectors from QR algorithm: \n{Q_final @ v}')

# Compare to NumPy
eigenvalues, eigenvectors = np.linalg.eig(A_original)
print(f'\nEigenvalues from NumPy: \n{eigenvalues}')
print(f'\nEigenvectors from NumPy: \n{eigenvectors}')

[-42.0239 -41.1004 -42.6385  0.8502 -0.0361  0.0942  0.0068  0.0009]
[-54.3942 -48.8944 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0378]
[-54.3835 -48.9051 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0377]
[-54.3834 -48.9052 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0377]
[-54.3834 -48.9052 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0377]
[-54.3834 -48.9052 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0377]

Eigenvectors from QR algorithm: 
[[-0.1853  0.2079  0.2574  0.3834  0.5393  0.3564 -0.4128 -0.0885]
 [-0.5309  0.4047  0.2861  0.1581  0.2906  0.3637 -0.4213 -0.3085]
 [-0.0312  0.1962  0.0413  0.3369  0.5134  0.4545 -0.4259 -0.1094]
 [-0.2760  0.5588  0.2974  0.3694  0.3603  0.1413 -0.2719 -0.4155]
 [-0.5249  0.4780  0.3161  0.3440  0.4335  0.0795 -0.3295 -0.5583]
 [-0.2318  0.1769  0.4512  0.4619  0.1416  0.1137 -0.3809 -0.2846]
 [-0.3945  0.2875  0.0537  0.1145  0.0623  0.3439 -0.2169 -0.0845]
 [-0.3490  0.3160  0.6748  0.4830  0.1397  0.6137 -0.3099 -0.5591]]

Eigenvalues fr

For convergence analysis of QR algorithm, see `Convergence of the LR, QR, and Related Algorithms` by J.H. Wilkinson